In [1]:
pip install folium


Note: you may need to restart the kernel to use updated packages.


In [3]:
import hvplot.pandas
import folium
from folium import plugins
import pandas as pd
import requests
import cartopy
import geoviews
import pyproj
import holoviews
import numpy as np
# Import API key
from api_keys import geoapify_key
from api_keys import weather_api_key

In [6]:

#load csv
poverty_expo_df= pd.read_csv("../Challenges/poverty_exposure.csv")

poverty_expo_df

,year,share_poverty_exposure,state_name,place_name
0,2021,0.450954,Alabama,Auburn city
1,2021,0.183112,Alabama,Auburn city
2,2021,0.731024,Alabama,Auburn city
3,2021,0.625397,Alabama,Auburn city
4,2021,0.513438,Alabama,Auburn city
...,...,...,...,...
2425,2021,0.099791,Wisconsin,Racine city
2426,2021,0.190358,Wisconsin,Racine city
2427,2021,0.052749,Wisconsin,Racine city
2428,2021,0.057825,Wisconsin,Racine city


In [7]:
poverty_expo_df = poverty_expo_df.rename(columns={
    "year": "year",
    "share_poverty_exposure": "poverty exposure",
    "state_name": "state",
    "place_name": "City"
})

poverty_expo_df = poverty_expo_df.drop_duplicates(subset="City", keep="first")
poverty_expo_df["City"] = poverty_expo_df["City"].str.rstrip("city")

poverty_expo_df

,year,poverty exposure,state,City
0,2021,0.450954,Alabama,Auburn
5,2021,0.341036,Alabama,Birmingham
10,2021,0.000000,Alabama,Hoover
15,2021,0.213182,Alabama,Huntsville
20,2021,0.155801,Alabama,Mobile
...,...,...,...,...
2405,2021,0.000000,Wisconsin,Green Bay
2410,2021,0.000000,Wisconsin,Kenosha
2415,2021,0.406296,Wisconsin,Madison
2420,2021,0.305411,Wisconsin,Milwaukee


In [8]:
import requests
import pandas as pd

# Extract the list of cities from the 'poverty_expo_df' DataFrame
cities = poverty_expo_df["City"].tolist()

# Create an empty DataFrame to store the data
city_location_data = pd.DataFrame(columns=["City", "Lat", "Lon"])

# Iterate through the list of cities
for city in cities:
    # Construct the API URL
    url = "http://api.openweathermap.org/data/2.5/weather"
    params = {"q": city, "units": "metric", "APPID": weather_api_key}
    
    # Make a request to the OpenWeather API
    response = requests.get(url, params=params)
    
    # Check if the response status code is OK
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        # Check if 'coord' key exists in the response
        if 'coord' in data:
            lat = data['coord'].get("lat")
            lon = data['coord'].get("lon")
            
            # Create a new DataFrame with the data for the current city
            city_data = pd.DataFrame({"City": [city], "Lat": [lat], "Lon": [lon]})
           
            # Concatenate the data with the city_location_data DataFrame
            city_location_data = pd.concat([city_location_data, city_data], ignore_index=True)
        else:
            print(f"Coordinates not found for {city}.")
    elif response.status_code == 404:
        print(f"City not found: {city}")
    else:
        print(f"Error fetching data for {city} with status code {response.status_code}.")

# Display the DataFrame with city, latitude, and longitude
city_location_data


City not found: Anchorage municipal
City not found: Gilbert town
City not found: Apple Valley town
City not found: Jurupa Valley 
City not found: San Buenaventura (Ventura) 
City not found: Davie town
City not found: Port St. Lucie 
City not found: St. Petersburg 
City not found: Athens-Clarke County unified government (balance)
City not found: Augusta-Richmond County consolidated government (balance)
City not found: Macon-Bibb Coun
City not found: Urban Honolulu CDP
City not found: Arlington Heights village
City not found: Cicero town
City not found: Schaumburg village
City not found: Indianapolis city (balance)
City not found: Lexington-Fayette urban coun
City not found: Louisville/Jefferson County metro government (balance)
City not found: Lee's Summit 
City not found: O'Fallon 
City not found: Cary town
City not found: Mount Pleasant town
City not found: Nashville-Davidson metropolitan government (balance)
City not found: Flower Mound town


,City,Lat,Lon
0,Auburn,47.3073,-122.2284
1,Birmingham,33.5207,-86.8025
2,Hoover,33.4054,-86.8114
3,Huntsville,34.7304,-86.5861
4,Mobile,30.6944,-88.0431
...,...,...,...
437,Green Bay,44.5192,-88.0198
438,Kenosha,42.5847,-87.8212
439,Madison,43.0731,-89.4012
440,Milwaukee,43.0389,-87.9065


In [9]:
city_location_data = pd.merge(city_location_data, poverty_expo_df, on="City")
city_location_data 

,City,Lat,Lon,year,poverty exposure,state
0,Auburn,47.3073,-122.2284,2021,0.450954,Alabama
1,Birmingham,33.5207,-86.8025,2021,0.341036,Alabama
2,Hoover,33.4054,-86.8114,2021,0.000000,Alabama
3,Huntsville,34.7304,-86.5861,2021,0.213182,Alabama
4,Mobile,30.6944,-88.0431,2021,0.155801,Alabama
...,...,...,...,...,...,...
437,Green Bay,44.5192,-88.0198,2021,0.000000,Wisconsin
438,Kenosha,42.5847,-87.8212,2021,0.000000,Wisconsin
439,Madison,43.0731,-89.4012,2021,0.406296,Wisconsin
440,Milwaukee,43.0389,-87.9065,2021,0.305411,Wisconsin


In [10]:

mm = folium.Map(location=[39.8283, -98.5795], zoom_start=3.4)
heat_data = [[row['Lat'], row['Lon'], row['poverty exposure']] for index, row in city_location_data.iterrows()]
folium.plugins.HeatMap(heat_data).add_to(mm)
mm

In [11]:
city_location_data.to_csv('city_poverty.csv', index=False)
